In [1]:
!pip install muax==0.0.2.7.1a0

  Attempting uninstall: muax
    Found existing installation: muax 0.0.2.7.1
    Uninstalling muax-0.0.2.7.1:
      Successfully uninstalled muax-0.0.2.7.1


In [1]:
import jax 
jax.config.update('jax_platform_name', 'cpu')

import gymnasium as gym 

# 1. Use `muax.fit` to fit CartPole-v1

In [2]:
import muax
from muax import nn 

`muax` provides example `representation`, `prediction` and `dynamic` modules

In [19]:
support_size = 10 
embedding_size = 8
discount = 0.99
num_actions = 2
full_support_size = int(support_size * 2 + 1)

repr_fn = nn._init_representation_func(nn.Representation, embedding_size)
pred_fn = nn._init_prediction_func(nn.Prediction, num_actions, full_support_size)
dy_fn = nn._init_dynamic_func(nn.Dynamic, embedding_size, num_actions, full_support_size)

Alternatively, you can use your customized models

In [20]:
from jax import numpy as jnp 
import haiku as hk


class Representation(hk.Module):
  def __init__(self, embedding_dim, name='representation'):
    super().__init__(name=name)

    self.repr_func = hk.Sequential([
        hk.Linear(embedding_dim)
    ])

  def __call__(self, obs):
    s = self.repr_func(obs)
    return s 


class Prediction(hk.Module):
  def __init__(self, num_actions, full_support_size, name='prediction'):
    super().__init__(name=name)        
    
    self.v_func = hk.Sequential([
        hk.Linear(16), jax.nn.elu,
        hk.Linear(full_support_size)
    ])
    self.pi_func = hk.Sequential([
        hk.Linear(16), jax.nn.elu,
        hk.Linear(num_actions)
    ])
  
  def __call__(self, s):
    v = self.v_func(s)
    logits = self.pi_func(s)
    logits = jax.nn.softmax(logits, axis=-1)
    return v, logits


class Dynamic(hk.Module):
  def __init__(self, embedding_dim, num_actions, full_support_size, name='dynamic'):
    super().__init__(name=name)
    
    self.ns_func = hk.Sequential([
        hk.Linear(16), jax.nn.elu,
        hk.Linear(embedding_dim)
    ])
    self.r_func = hk.Sequential([
        hk.Linear(16), jax.nn.elu,
        hk.Linear(full_support_size)
    ])
    self.cat_func = jax.jit(lambda s, a: 
                            jnp.concatenate([s, jax.nn.one_hot(a, num_actions)],
                                            axis=1)
                            )
  
  def __call__(self, s, a):
    sa = self.cat_func(s, a)
    r = self.r_func(sa)
    ns = self.ns_func(sa)
    return r, ns


def init_representation_func(representation_module, embedding_dim):
    def representation_func(obs):
      repr_model = representation_module(embedding_dim)
      return repr_model(obs)
    return representation_func
  
def init_prediction_func(prediction_module, num_actions, full_support_size):
  def prediction_func(s):
    pred_model = prediction_module(num_actions, full_support_size)
    return pred_model(s)
  return prediction_func

def init_dynamic_func(dynamic_module, embedding_dim, num_actions, full_support_size):
  def dynamic_func(s, a):
    dy_model = dynamic_module(embedding_dim, num_actions, full_support_size)
    return dy_model(s, a)
  return dynamic_func 

repr_fn = init_representation_func(Representation, embedding_size)
pred_fn = init_prediction_func(Prediction, 2, full_support_size)
dy_fn = init_dynamic_func(Dynamic, embedding_size, 2, full_support_size)  
    

`muax` has `Episode tracer` and `replay buffuer` to track and store trajectories from interacting with environments

In [21]:
tracer = muax.PNStep(10, discount, 0.5)
buffer = muax.TrajectoryReplayBuffer(500)

`muax` leverages `optax` to update weights

In [22]:
gradient_transform = muax.model.optimizer(init_value=0.02, peak_value=0.02, end_value=0.002, warmup_steps=5000, transition_steps=5000)
# gradient_transform = optax.adam(0.02)

In [23]:
model = muax.MuZero(repr_fn, pred_fn, dy_fn, policy='muzero', discount=discount,
                    optimizer=gradient_transform, support_size=support_size)

model_path = muax.fit(model, 'CartPole-v1', 
                    max_episodes=1000,
                    max_training_steps=10000,
                    tracer=tracer,
                    buffer=buffer,
                    k_steps=10,
                    sample_per_trajectory=1,
                    num_trajectory=32,
                    tensorboard_dir='/home/fangbowen/tensorboard/cartpole',
                    model_save_path='/home/fangbowen/models/cartpole',
                    save_name='cartpole_model_params',
                    random_seed=0,
                    log_all_metrics=True)


buffer warm up stage...
start training...


INFO:TrainMonitor:ep: 1,	T: 9,	G: 8,	avg_r: 1,	avg_G: 8,	t: 8,	dt: 2443.100ms,	v: 0.0416,	Rn: 4.4,	loss: 4.32,	training_step: 7,	test_G: 8.8
INFO:TrainMonitor:ep: 2,	T: 18,	G: 8,	avg_r: 1,	avg_G: 8,	t: 8,	dt: 1725.632ms,	v: 5.42,	Rn: 4.4,	loss: 2.11,	training_step: 14
INFO:TrainMonitor:ep: 3,	T: 39,	G: 20,	avg_r: 1,	avg_G: 12,	t: 20,	dt: 460.329ms,	v: 7.01,	Rn: 10.7,	loss: 1.88,	training_step: 33
INFO:TrainMonitor:ep: 4,	T: 49,	G: 9,	avg_r: 1,	avg_G: 11.2,	t: 9,	dt: 13.108ms,	v: 8.9,	Rn: 4.87,	loss: 1.77,	training_step: 41
INFO:TrainMonitor:ep: 5,	T: 59,	G: 9,	avg_r: 1,	avg_G: 10.8,	t: 9,	dt: 13.046ms,	v: 7.71,	Rn: 4.87,	loss: 1.75,	training_step: 49
INFO:TrainMonitor:ep: 6,	T: 79,	G: 19,	avg_r: 1,	avg_G: 12.2,	t: 19,	dt: 160.603ms,	v: 8.71,	Rn: 10.7,	loss: 1.72,	training_step: 67
INFO:TrainMonitor:ep: 7,	T: 100,	G: 20,	avg_r: 1,	avg_G: 13.3,	t: 20,	dt: 12.565ms,	v: 9.77,	Rn: 11.8,	loss: 1.66,	training_step: 86
INFO:TrainMonitor:ep: 8,	T: 114,	G: 13,	avg_r: 1,	avg_G: 13.2,	t: 13,	dt: 1

In [24]:
model_path

'/home/fangbowen/models/epoch_0220_test_G_287.80000000/cartpole_model_params'

In [25]:
model = muax.MuZero(repr_fn, pred_fn, dy_fn, policy='muzero', discount=discount,
                    optimizer=gradient_transform, support_size=support_size)

model.load(model_path)

In [26]:
from muax.test import test
env_id = 'CartPole-v1'
test_env = gym.make(env_id, render_mode='rgb_array')
test_key = jax.random.PRNGKey(0)
test(model, test_env, test_key, num_simulations=50, num_test_episodes=100, random_seed=None)

255.24

# 2. Customize

## 2.1 Customize the neural networks

`muax` uses `haiku` to implement the neural networks. A tutorial for using `haiku` can be found at (link). 

In [8]:
from jax import numpy as jnp 
import haiku as hk


class Representation(hk.Module):
  def __init__(self, embedding_dim, name='representation'):
    super().__init__(name=name)

    self.repr_func = hk.Sequential([
        hk.Linear(embedding_dim), jax.nn.elu
    ])

  def __call__(self, obs):
    s = self.repr_func(obs)
    return s 


class Prediction(hk.Module):
  def __init__(self, num_actions, full_support_size, name='prediction'):
    super().__init__(name=name)        
    
    self.v_func = hk.Sequential([
        hk.Linear(16), jax.nn.elu,
        hk.Linear(full_support_size)
    ])
    self.pi_func = hk.Sequential([
        hk.Linear(16), jax.nn.elu,
        hk.Linear(num_actions)
    ])
  
  def __call__(self, s):
    v = self.v_func(s)
    logits = self.pi_func(s)
    logits = jax.nn.softmax(logits, axis=-1)
    return v, logits


class Dynamic(hk.Module):
  def __init__(self, embedding_dim, num_actions, full_support_size, name='dynamic'):
    super().__init__(name=name)
    
    self.ns_func = hk.Sequential([
        hk.Linear(16), jax.nn.elu,
        hk.Linear(embedding_dim)
    ])
    self.r_func = hk.Sequential([
        hk.Linear(16), jax.nn.elu,
        hk.Linear(full_support_size)
    ])
    self.cat_func = jax.jit(lambda s, a: 
                            jnp.concatenate([s, jax.nn.one_hot(a, num_actions)],
                                            axis=1)
                            )
  
  def __call__(self, s, a):
    sa = self.cat_func(s, a)
    r = self.r_func(sa)
    ns = self.ns_func(sa)
    return r, ns


def init_representation_func(representation_module, embedding_dim):
    def representation_func(obs):
      repr_model = representation_module(embedding_dim)
      return repr_model(obs)
    return representation_func
  
def init_prediction_func(prediction_module, num_actions, full_support_size):
  def prediction_func(s):
    pred_model = prediction_module(num_actions, full_support_size)
    return pred_model(s)
  return prediction_func

def init_dynamic_func(dynamic_module, embedding_dim, num_actions, full_support_size):
  def dynamic_func(s, a):
    dy_model = dynamic_module(embedding_dim, num_actions, full_support_size)
    return dy_model(s, a)
  return dynamic_func 
    

In [9]:
support_size = 10 
embedding_size = 8
full_support_size = int(support_size * 2 + 1)
repr_fn = init_representation_func(Representation, embedding_size)
pred_fn = init_prediction_func(Prediction, 2, full_support_size)
dy_fn = init_dynamic_func(Dynamic, embedding_size, 2, full_support_size)

## 2.2 Customize the training loop

inside the `muax.fit` function, the main structure is a typical RL interacting loop. Reset the env, agent takes an action based on the observation, updated current state until done

In [10]:
import numpy as np 
from muax import Trajectory

def temperature_fn(max_training_steps, training_steps):
  if training_steps < 0.5 * max_training_steps:
      return 1.0
  elif training_steps < 0.75 * max_training_steps:
      return 0.5
  else:
      return 0.25
  
def test(model, env, key, num_simulations, num_test_episodes=10, random_seed=None):
    total_rewards = np.zeros(num_test_episodes)
    for episode in range(num_test_episodes):
        obs, info = env.reset(seed=random_seed)
        done = False
        episode_reward = 0
        for t in range(env.spec.max_episode_steps):
            key, subkey = jax.random.split(key)
            a = model.act(subkey, obs, 
                          with_pi=False, 
                          with_value=False, 
                          obs_from_batch=False,
                          num_simulations=num_simulations,
                          temperature=0.) # Use deterministic actions during testing
            obs_next, r, done, truncated, info = env.step(a)
            episode_reward += r
            if done or truncated:
                break 
            obs = obs_next 
        
        total_rewards[episode] = episode_reward

    average_test_reward = np.mean(total_rewards)
    return average_test_reward  

In [11]:
random_seed = 0
discount = 0.997
buffer_warm_up = 64
max_training_steps = 10000
max_episodes = 1000
num_simulations = 50
num_test_episodes = 10
num_trajectory = 32
sample_per_trajectory = 1
k_steps = 10

gradient_transform = muax.model.optimizer(init_value=0.02, peak_value=0.02, end_value=0.002, warmup_steps=5000, transition_steps=5000)
tracer = muax.PNStep(10, discount, 0.5)
buffer = muax.TrajectoryReplayBuffer(500)
model = muax.MuZero(repr_fn, pred_fn, dy_fn, optimizer=gradient_transform, discount=discount)

env_id = 'CartPole-v1'
env = gym.make(env_id, render_mode='rgb_array')
test_env = gym.make(env_id, render_mode='rgb_array')

sample_input = jnp.expand_dims(jnp.zeros(env.observation_space.shape), axis=0)
key = jax.random.PRNGKey(random_seed)
key, test_key, subkey = jax.random.split(key, num=3)
model.init(subkey, sample_input) 

training_step = 0
best_test_G = -float('inf')
max_training_steps_reached = False

# buffer warm up
print('buffer warm up stage...')
while len(buffer) < buffer_warm_up:
  obs, info = env.reset()    
  trajectory = Trajectory()
  temperature = temperature_fn(max_training_steps=max_training_steps, training_steps=training_step)
  for t in range(env.spec.max_episode_steps):
    key, subkey = jax.random.split(key)
    a, pi, v = model.act(subkey, obs, 
                          with_pi=True, 
                          with_value=True, 
                          obs_from_batch=False,
                          num_simulations=num_simulations,
                          temperature=temperature)
    obs_next, r, done, truncated, info = env.step(a)
    if truncated:
      r = 1 / (1 - tracer.gamma)
    tracer.add(obs, a, r, done or truncated, v=v, pi=pi)
    while tracer:
      trans = tracer.pop()
      trajectory.add(trans)
    if done or truncated:
      break 
    obs = obs_next 
  trajectory.finalize()
  if len(trajectory) >= k_steps:
    buffer.add(trajectory, trajectory.batched_transitions.w.mean())
  
print('start training...')
  
for ep in range(max_episodes):
  obs, info = env.reset(seed=random_seed)    
  trajectory = Trajectory()
  temperature = temperature_fn(max_training_steps=max_training_steps, training_steps=training_step)
  for t in range(env.spec.max_episode_steps):
    key, subkey = jax.random.split(key)
    a, pi, v = model.act(subkey, obs, 
                          with_pi=True, 
                          with_value=True, 
                          obs_from_batch=False,
                          num_simulations=num_simulations,
                          temperature=temperature)
    obs_next, r, done, truncated, info = env.step(a)
    if truncated:
      r = 1 / (1 - tracer.gamma)
    tracer.add(obs, a, r, done or truncated, v=v, pi=pi)
    while tracer:
      trans = tracer.pop()
      trajectory.add(trans)
    if done or truncated:
      break 
    obs = obs_next 
  trajectory.finalize()
  if len(trajectory) >= k_steps:
    buffer.add(trajectory, trajectory.batched_transitions.w.mean())
  
  if max_training_steps_reached:
    break
  train_loss = 0
  for _ in range(t):
    transition_batch = buffer.sample(num_trajectory=num_trajectory,
                                      sample_per_trajectory=sample_per_trajectory,
                                      k_steps=k_steps)
    loss_metric = model.update(transition_batch)
    train_loss += loss_metric['loss']
    training_step += 1
    if training_step >= max_training_steps:
      max_training_steps_reached = True
      break
  train_loss /= t
  print(f'epoch: {ep:04d}, loss: {train_loss:.8f}, training_step: {training_step}')

  test_G = test(model, test_env, test_key, num_simulations=num_simulations, num_test_episodes=num_test_episodes)      
  print(f'epoch: {ep:04d}, test_G: {test_G:.8f}')
  if test_G >= best_test_G:
    best_test_G = test_G

print(f'Best total reward in test: {best_test_G}')

buffer warm up stage...
start training...
epoch: 0000, loss: 4.30174676, training_step: 8
epoch: 0000, test_G: 9.20000000
epoch: 0001, loss: 2.33003187, training_step: 15
epoch: 0001, test_G: 9.20000000
epoch: 0002, loss: 2.13727483, training_step: 22
epoch: 0002, test_G: 9.10000000
epoch: 0003, loss: 2.19564544, training_step: 31
epoch: 0003, test_G: 10.00000000
epoch: 0004, loss: 2.18947538, training_step: 38
epoch: 0004, test_G: 9.20000000
epoch: 0005, loss: 2.07392795, training_step: 45
epoch: 0005, test_G: 9.20000000
epoch: 0006, loss: 2.12085823, training_step: 55
epoch: 0006, test_G: 10.10000000
epoch: 0007, loss: 2.04124657, training_step: 62
epoch: 0007, test_G: 21.10000000
epoch: 0008, loss: 2.10093344, training_step: 85
epoch: 0008, test_G: 9.40000000
epoch: 0009, loss: 2.07039206, training_step: 96
epoch: 0009, test_G: 9.30000000
epoch: 0010, loss: 1.95028911, training_step: 103
epoch: 0010, test_G: 32.60000000
epoch: 0011, loss: 2.03606952, training_step: 117
epoch: 0011, 